In [1]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs

In [2]:
import pandas as pd 
import numpy as np
df = pd.read_csv('movie_data2.csv')
display(df)

,Series_Title,Informations
0,The Shawshank Redemption,Two imprisoned men bond over a number of years...
1,The Godfather,An organized crime dynasty's aging patriarch t...
2,The Dark Knight,When the menace known as the Joker wreaks havo...
3,The Godfather: Part II,The early life and career of Vito Corleone in ...
4,12 Angry Men,A jury holdout attempts to prevent a miscarria...
...,...,...
995,Breakfast at Tiffany's,A young New York socialite becomes interested ...
996,Giant,Sprawling epic covering the life of a Texas ca...
997,From Here to Eternity,"In Hawaii in 1941, a private is cruelly punish..."
998,Lifeboat,Several survivors of a torpedoed merchant ship...


In [3]:
Informations = tf.data.Dataset.from_tensor_slices(dict(df)).\
    map(lambda x: {
    "Informations": x["Informations"],
    "Series_Title": x["Series_Title"],
})

In [4]:
for x in Informations.take(10).as_numpy_iterator():
  pprint.pprint(x)

{'Informations': b'Two imprisoned men bond over a number of years, finding sola'
                 b'ce and eventual redemption through acts of common decency.',
 'Series_Title': b'The Shawshank Redemption'}
{'Informations': b"An organized crime dynasty's aging patriarch transfers contr"
                 b'ol of his clandestine empire to his reluctant son.',
 'Series_Title': b'The Godfather'}
{'Informations': b'When the menace known as the Joker wreaks havoc and chaos on'
                 b' the people of Gotham, Batman must accept one of the greates'
                 b't psychological and physical tests of his ability to fight i'
                 b'njustice.',
 'Series_Title': b'The Dark Knight'}
{'Informations': b'The early life and career of Vito Corleone in 1920s New York'
                 b' City is portrayed, while his son, Michael, expands and tigh'
                 b'tens his grip on the family crime syndicate.',
 'Series_Title': b'The Godfather: Part II'}
{'Informations': b'A j

In [5]:
Series_Title = tf.data.Dataset.from_tensor_slices(dict(df)).\
    map(lambda x: x["Series_Title"],
)

In [6]:
for x in Series_Title.take(10).as_numpy_iterator():
  pprint.pprint(x)

b'The Shawshank Redemption'
b'The Godfather'
b'The Dark Knight'
b'The Godfather: Part II'
b'12 Angry Men'
b'The Lord of the Rings: The Return of the King'
b'Pulp Fiction'
b"Schindler's List"
b'Inception'
b'Fight Club'


In [7]:
train = Informations.take(1000)
test = Informations.skip(800).take(200)

In [8]:
Series_Title2 = Series_Title.batch(1000)
Informations2 = Informations.batch(1000).map(lambda x: x["Informations"])

unique_movie_titles = np.unique(np.concatenate(list(Series_Title2)))
unique_Informations = np.unique(np.concatenate(list(Informations2)))

unique_Informations[:10]

array([b'"Documentary" about a man who can look and act like whoever he\'s around, and meets various famous people.',
       b'"The Washington Post" reporters Bob Woodward and Carl Bernstein uncover the details of the Watergate scandal that leads to President Richard Nixon\'s resignation.',
       b"1183 A.D.: King Henry II's three sons all want to inherit the throne, but he won't commit to a choice. They and his wife variously plot to force him.",
       b'160 elite U.S. soldiers drop into Somalia to capture two top lieutenants of a renegade warlord and find themselves in a desperate battle with a large force of heavily-armed Somalis.',
       b'24 hours in the lives of three young men in the French suburbs the day after a violent riot.',
       b'78-year-old Carl Fredricksen travels to Paradise Falls in his house equipped with balloons, inadvertently taking a young stowaway.',
       b'A 20-something supervising staff member of a residential treatment facility navigates the troubled 

In [9]:
embedding_dimension = 32

In [10]:
Informations_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_Informations, mask_token=None),
  # additional embedding for unknown tokens
  tf.keras.layers.Embedding(len(unique_Informations) + 1, embedding_dimension)
])

In [11]:
Series_Title_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

In [12]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=Series_Title.batch(128).map(Series_Title_model)
)

In [13]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [14]:
class SimilarityModel(tfrs.Model):

  def __init__(self, Informations_model, Series_Title_model):
    super().__init__()
    self.Series_Title_model: tf.keras.Model = Series_Title_model
    self.Informations_model: tf.keras.Model = Informations_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    #  pick out the informations features and pass them into the informations model.
    Informations_embeddings = self.Informations_model(features["Informations"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    Series_Title_model_embeddings = self.Series_Title_model(features["Series_Title"])

    # The task computes the loss and the metrics.
    return self.task(Informations_embeddings, Series_Title_model_embeddings)

In [15]:
model = SimilarityModel(Informations_model, Series_Title_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [16]:
cached_train = train.shuffle(1000).batch(500).cache()
cached_test = test.batch(500).cache()

In [19]:
model.fit(cached_train, epochs=500)

Epoch 1/500
2/2 [==============================] - 0s 132ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.9990 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 3012.9801 - regularization_loss: 0.0000e+00 - total_loss: 3012.9801
Epoch 2/500
2/2 [==============================] - 0s 125ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.9990 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 2953.9556 - regularization_loss: 0.0000e+00 - total_loss: 2953.9556
Epoch 3/500
2/2 [==============================] - 0s 139ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_ac

2/2 [==============================] - 0s 166ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 61.8489 - regularization_loss: 0.0000e+00 - total_loss: 61.8489
Epoch 21/500
2/2 [==============================] - 0s 117ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 54.3665 - regularization_loss: 0.0000e+00 - total_loss: 54.3665
Epoch 22/500
2/2 [==============================] - 0s 254ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - f

2/2 [==============================] - 0s 118ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 16.4217 - regularization_loss: 0.0000e+00 - total_loss: 16.4217
Epoch 40/500
2/2 [==============================] - 0s 126ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 15.7992 - regularization_loss: 0.0000e+00 - total_loss: 15.7992
Epoch 41/500
2/2 [==============================] - 0s 126ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - f

2/2 [==============================] - 0s 163ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 9.4355 - regularization_loss: 0.0000e+00 - total_loss: 9.4355
Epoch 59/500
2/2 [==============================] - 0s 145ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 9.2326 - regularization_loss: 0.0000e+00 - total_loss: 9.2326
Epoch 60/500
2/2 [==============================] - 0s 132ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - facto

2/2 [==============================] - 0s 108ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 6.7024 - regularization_loss: 0.0000e+00 - total_loss: 6.7024
Epoch 78/500
2/2 [==============================] - 0s 112ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 6.6046 - regularization_loss: 0.0000e+00 - total_loss: 6.6046
Epoch 79/500
2/2 [==============================] - 0s 123ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - facto

...

2/2 [==============================] - 0s 110ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 1.6770 - regularization_loss: 0.0000e+00 - total_loss: 1.6770
Epoch 420/500
2/2 [==============================] - 0s 107ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 1.6749 - regularization_loss: 0.0000e+00 - total_loss: 1.6749
Epoch 421/500
2/2 [==============================] - 0s 109ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - fac

2/2 [==============================] - 0s 110ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 1.6400 - regularization_loss: 0.0000e+00 - total_loss: 1.6400
Epoch 439/500
2/2 [==============================] - 0s 123ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 1.6381 - regularization_loss: 0.0000e+00 - total_loss: 1.6381
Epoch 440/500
2/2 [==============================] - 0s 110ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - fac

2/2 [==============================] - 0s 124ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 1.6063 - regularization_loss: 0.0000e+00 - total_loss: 1.6063
Epoch 458/500
2/2 [==============================] - 0s 111ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 1.6046 - regularization_loss: 0.0000e+00 - total_loss: 1.6046
Epoch 459/500
2/2 [==============================] - 0s 110ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - fac

2/2 [==============================] - 0s 109ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 1.5755 - regularization_loss: 0.0000e+00 - total_loss: 1.5755
Epoch 477/500
2/2 [==============================] - 0s 121ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 1.5740 - regularization_loss: 0.0000e+00 - total_loss: 1.5740
Epoch 478/500
2/2 [==============================] - 0s 110ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - fac

2/2 [==============================] - 0s 118ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 1.5473 - regularization_loss: 0.0000e+00 - total_loss: 1.5473
Epoch 496/500
2/2 [==============================] - 0s 108ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 1.5459 - regularization_loss: 0.0000e+00 - total_loss: 1.5459
Epoch 497/500
2/2 [==============================] - 0s 124ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - fac

In [18]:
model.evaluate(cached_test, return_dict=True)

1/1 [==============================] - 1s 742ms/step - factorized_top_k/top_1_categorical_accuracy: 0.7500 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 1022.1049 - regularization_loss: 0.0000e+00 - total_loss: 1022.1049


{'factorized_top_k/top_1_categorical_accuracy': 0.75,
 'factorized_top_k/top_5_categorical_accuracy': 1.0,
 'factorized_top_k/top_10_categorical_accuracy': 1.0,
 'factorized_top_k/top_50_categorical_accuracy': 1.0,
 'factorized_top_k/top_100_categorical_accuracy': 1.0,
 'loss': 1022.1048583984375,
 'regularization_loss': 0,
 'total_loss': 1022.1048583984375}

In [66]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.Informations_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((Series_Title.batch(100), Series_Title.batch(100).map(model.Series_Title_model)))
)

# Get recommendations.
_, titles = index(tf.constant(["Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency."]))
print(f"Recommendations for this movie description : {titles[0, :10]}")
print(f"The score obtain for each movie: {scores[0][:10]}")

Recommendations for this movie description : [b'The Shawshank Redemption' b'Kai po che!' b'Lord of War'
 b'Dazed and Confused' b'American Beauty' b'Scent of a Woman'
 b'Diarios de motocicleta' b'Andaz Apna Apna' b'Ford v Ferrari'
 b'Rash\xc3\xb4mon']
The score obtain for each movie: [17.037783   8.625467   8.025611   7.299474   6.932453   6.932435
  6.7647905  6.6890106  6.533307   6.4087253]


In [69]:
# Export the query model.
with tempfile.TemporaryDirectory() as tmp:
  path = os.path.join(tmp, "model")

  # Save the index.
  tf.saved_model.save(index, path)

  # Load it back; can also be done in TensorFlow Serving.
  loaded = tf.saved_model.load(path)

  # Pass a movie description in, get top predicted movie titles back.
  scores, titles = loaded(["When his only friend dies, a man born with dwarfism moves to rural New Jersey to live a life of solitude, only to meet a chatty hot dog vendor and a woman dealing with her own personal loss."])

  print(f"Recommendations for this movie description: {titles[0][:10]}")
print(f"The score obtain for each movie: {scores[0][:10]}")

INFO:tensorflow:Assets written to: C:\Users\Acer\AppData\Local\Temp\tmps3b3ef0e\model\assets


INFO:tensorflow:Assets written to: C:\Users\Acer\AppData\Local\Temp\tmps3b3ef0e\model\assets


Recommendations for this movie description: [b'The Station Agent' b'Scarface' b'Raazi' b'Barton Fink'
 b'La Grande Illusion' b'Mr. Smith Goes to Washington' b'Toy Story'
 b'La piel que habito' b'Star Wars: Episode VII - The Force Awakens'
 b'Kakushi-toride no san-akunin']
The score obtain for each movie: [15.868319  11.406765   8.130211   7.0137377  6.6616354  6.5723734
  6.43346    6.313094   6.310127   6.0390673]


In [70]:
tf.saved_model.save(index, 'C:\\Users\\Acer\\Desktop\\Movie data\\SimilarityModel')

INFO:tensorflow:Assets written to: C:\Users\Acer\Desktop\Movie data\SimilarityModel\assets


INFO:tensorflow:Assets written to: C:\Users\Acer\Desktop\Movie data\SimilarityModel\assets


In [71]:
converter = tf.lite.TFLiteConverter.from_saved_model('C:\\Users\\Acer\\Desktop\\Movie data\\SimilarityModel')
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert()

# Save the model.
with open('SimilarityModel.tflite', 'wb') as f:
  f.write(tflite_model)